In [82]:
import pandas as py
import matplotlib.pyplot as pt
import numpy as np
from sklearn.linear_model import LogisticRegression
train = py.read_csv('train.csv')
print(train_df.head())
test = py.read_csv('Test.csv')

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [38]:
train.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [96]:
train_ = train.copy()
train_["Age"].fillna(train["Age"].median(skipna=True), inplace=True)
train_.head

<bound method NDFrame.head of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                     

In [97]:
train_['Sex']=np.where(train_['Sex']=='female',0,1)
train_.head  



<bound method NDFrame.head of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    1  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1   
2                               Heikkinen, Miss. Laina    0  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1   
4                             Allen, Mr. William Henry    1  35.0      0   
..                                                 ...  .

In [88]:
X = train_[['Pclass','Sex','Age','SibSp','Fare','Parch']].values
y = train_['Survived'].values       

In [98]:
print(X.shape)
print(y.shape)


(891, 6)
(891,)


In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [100]:
X_train[0]

array([ 1. ,  0. , 30. ,  0. , 93.5,  0. ])

In [101]:
model = LogisticRegression(solver='liblinear')

In [102]:
M = model.fit(X_train,y_train)

In [103]:
print(model.score(X,y))

0.7991021324354658


In [110]:
# 0 means that the person will not survive and 1 means he will .
print(model.predict([[1 , 1 ,60 , 0 , 30 , 0]]))

[0]


In [105]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print(model.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))

LogisticRegression accuracy is 0.772
